In [3]:
# Package Imports and CSV Reads

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import time
import tqdm

import warnings
warnings.filterwarnings("ignore")

only_23_all = pd.read_csv('player_lists/only_23_all.csv')
only_23_gk = pd.read_csv('player_lists/only_23_gk.csv')
pre_23_all = pd.read_csv('player_lists/pre_23_all.csv')
pre_23_gk = pd.read_csv('player_lists/pre_23_gk.csv')



from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#This will get you player name - need to use in scraper function

test_player = 'https://fbref.com/en/players/f07be45a/matchlogs/2018/summary/Wayne-Rooney-Match-Logs'
response = requests.get(test_player)
soup = BeautifulSoup(response.text,'html.parser')
test_div = soup.find('div', id='meta')
test_div.find('span').text

In [4]:
cats = ['summary','passing','passing_types','gca','defense','possession','misc']

In [5]:
summary_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Gls','Ast','PK Made','PKatt','Sh','SoT','CrdY','CrdR','Touches','Tkl','Int','Blocks','xG','npxG','xAG','SCA','GCA','Passes Completed','Passes Attempted','Passing Cmp%','Progressive Passes','Carries','Progressive Carries','Take on Att','Take on Success','Match Report']]

passing_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Total Passes Completed','Total Passes Attempted','Total Passing Completion %','Total Passing Distance','Total Progressive Passing Distance','Short Passes Completed','Short Passes Attempted','Short Passing Cmp%','Medium Passes Completed','Medium Passes Attempted','Medium Passes Completion %','Long Passes Complete','Long Passes Attempted','Long Pass Completion %','Assists','xAG','expected assists','Key Passes','Passes into final third','Passes into penalty area','Crosses into penalty','Progressive Passes','Match Report']]

pass_type_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Passes Attempted','Live Ball Passes','Dead-ball passes','Passes from FK','Through Balls','Switch Passes','Crosses','Throw-ins','Corner Kicks','In-swinging Corner','Outswinging Corner','Straight Corner','Passes Complete','Passes Offside','Passes Blocked','Match Report']]

gca_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','SCA','SCA from Live Pass','SCA from deadball pass','SCA from Take on','SCA from Shot','SCA from foul drawn','SCA from defense action','GCA','GCA from Live Pass','GCA from deadball pass','GCA from Take on','GCA from Shot','GCA from foul drawn','GCA from defense action','Match Report']]

defense_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Tackles','Tackles Won','Tackles in defensive third','Tackles in Mid third','Tackles in Attacking third','Dribblers Tackled','Attempted Tackles on Dribbler','% of succesfull tackles on dribbler','Unsuccesful tackles on dribbler','Blocks','Shots Blocked','Passes Blocked','Interceptions','Tackles + Interceptions','Clearances','Errors','Match Report']]

poss_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Touches','Touches in Def Pen Area','Touches in Def Third','Touches in Mid Third','Touches in Att Third','Touches in Attacking Penalty Box','Live Ball Touches','Attempted Take On','Succesful Take On','Take on Success Rate','Times Tackled During Take On','Tackled During Take on %','Carries','Total Carry Distance','Prg Carry Distance','Carries into final third','Carries into penalty box','Miscontrols','Times Dispossesed','Passes Received','Progressive Passes Received','Match Report']]

misc_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Yellow Cards','Red Cards','2nd Yellow Card','Fouls','Fouls Drawn','Offsides','Crosses','Interceptions','Tackles Won','Penalty Kicks Won','Penalty Kicks Conceded','Own Goals','Ball Recoveries','Aerials Won','Aerieals Lost','Aerial Duel win %','Match Report']]


In [23]:
#Player Scraper Function

def player_scraper(link_list,dataframe,column_list,csv_title):
    i = 0
    for link in tqdm.tqdm(link_list):
        time.sleep(3)
        
        response = requests.get(link_list[i])
        soup = BeautifulSoup(response.text,'html.parser')
        name_div = soup.find('div', id='meta')
        player_name = name_div.find('span').text
        i += 1

        data_table = soup.find('table',{'id': 'matchlogs_all'})
        table_body = data_table.find('tbody')
        table_rows = table_body.find_all('tr')

        data = []

        for row in table_rows:
                cols = row.find_all(['td','th'])
                cols = [col.text.strip() for col in cols]
                data.append(cols)
                
        temp_df = pd.DataFrame(data, columns=column_list)
        temp_df['Player_Name'] = player_name
        dataframe = dataframe.append(temp_df, ignore_index=True)
    dataframe.to_csv(f"player_data/{csv_title}.csv",index=False)
    return 

In [21]:
summary_df = pd.DataFrame(columns=summary_columns, dtype=object)

In [7]:
test_summary_df = pd.DataFrame(columns=summary_columns, dtype=object)

In [8]:
mls_2023_players = pd.read_csv('player_lists/mls_field/test_list.csv')

In [9]:
test_list = pd.read_csv('player_lists/mls_field/test_list.csv')
test_list = test_list.dropna()
test_list['Year'] = test_list['Year'].astype(int)

In [ ]:
test_list

In [ ]:
test_list.info()

In [ ]:
range(len(list(test_list['Squad_Code'])))

In [ ]:
test_list['Squad_Code'][0]

In [15]:
#Link Creation - Creating list of all links before 2023 and one list of 2023 for both big 5 & MLS.

##
mls_links = []
for category in range(len(cats)):
    for squad in range(len(list(test_list['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{test_list['Squad_Code'][squad]}/matchlogs/{test_list['Year'][squad]}/{cats[category]}/{test_list['Text'][squad]}")
        mls_links.append(link)

field_summary_23 = []
for squad in range(len(list(test_list['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{test_list['Squad_Code'][squad]}/matchlogs/{test_list['Year'][squad]}/summary/{test_list['Text'][squad]}")
        field_summary_23.append(link)
        
mls_field_passing_23 = []
for squad in range(len(list(test_list['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{test_list['Squad_Code'][squad]}/matchlogs/{test_list['Year'][squad]}/summary/{test_list['Text'][squad]}")
        mls_field_passing_23.append(link)
        
gk_before_23_summary = []



In [17]:
len(field_summary_23)

20

In [18]:
field_summary_23[0]

'https://fbref.com/en/players/a79b9e17/matchlogs/2023/summary/Njabulo-Blom-Match-Logs'

In [ ]:
# Test Scrape of Summary DF, convert to function after. 
i = 0
for link in tqdm.tqdm(mls_field_summary_23):
    time.sleep(3)

    response = requests.get(mls_field_summary_23[i])
    soup = BeautifulSoup(response.text,'html.parser')
    name_div = soup.find('div', id='meta')
    player_name = name_div.find('span').text
    i += 1
    
    data_table = soup.find('table',{'id': 'matchlogs_all'})
    table_body = data_table.find('tbody')
    table_rows = table_body.find_all('tr')
    
    data = []
    
    for row in table_rows:
            cols = row.find_all(['td','th'])
            cols = [col.text.strip() for col in cols]
            data.append(cols)
    
    temp_df = pd.DataFrame(data, columns=summary_columns)
    temp_df['Player_Name'] = player_name
    summary_df = summary_df.append(temp_df, ignore_index=True)
    

In [ ]:
summary_df.head()

In [ ]:
summary_df['Player_Name']

In [19]:
test = field_summary_23[0:3]

In [24]:
player_scraper(test,summary_df,summary_columns,"Summary")

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:10<00:00,  3.36s/it]


In [ ]:
summary_df